# DEM Download, Co-Registration, and Differencing for Geodetic Glacier Mass Balance Analysis 

### This Python workflow will walk you through the steps to: 

1. Pull the SRTM DEM corresponding to your study area from EarthAccess -OR- load an existing reference DEM of your choosing 
2. Load the additional DEM(s) you wish to co-register 
3. Perform DEM co-registration, following the workflow of Nuth and Kääb (2011)
4. Clip the co-registered DEMs to the glacier area of your choosing (RGI, GLIMS, DGA, etc.) 
5. Difference the DEMs to compute volume loss within glacier area 
6. Calculate geodetic glacier mass balance 
7. Produce figures  

This workflow has the following requirements: 
- a NASA EarthData login to download the SRTM elevation data. EarthData can be accessed at the following link: (https://urs.earthdata.nasa.gov/oauth/authorize?client_id=ZAQpxSrQNpk342OR77kisA&response_type=code&redirect_uri=https://appeears.earthdatacloud.nasa.gov/login&state=/)
- additional DEMs of the study area (e.g. UAV or satellite-derived product)

We use `earthaccess`, an open source package developed by Luis Lopez to allow easy search of the NASA Common Metadata Repository (CMR) and download of NASA data collections. It can be used for programmatic search and access for both _DAAC-hosted_ and _cloud-hosted_ data. It manages authenticating using Earthdata Login credentials. `earthaccess` can be used to find and access both DAAC-hosted and cloud-hosted data in just **three** lines of code.  See [https://github.com/nsidc/earthaccess](https://github.com/nsidc/earthaccess).

## Step 1: Pull SRTM for study area as reference DEM 

In [ ]:
#Download SRTM DEM from EarthAccess

# For searching and accessing NASA data
import earthaccess
import os

auth = earthaccess.login()
# you will be prompted to enter your EarthAccess login and password 

# search for SRTM data
query = earthaccess.search_datasets(
            keyword="SRTM",
)

results = earthaccess.search_data(
    short_name = 'SRTMGL1', # SRTM 1-arcsecond Global dataset (confirm this is the right SRTM dataset...)
    cloud_hosted = True, # Ensure access to cloud-stored data
    bounding_box = (-134.7,58.9,-133.9,59.2), #update the bounding box for your study area 
        # box format is lon, lat, for bottom left then top right
        # example bounding box is for the Nevados de Chillán volcanic complex in Ñuble, Chile 
)

# Define output directory
output_dir = "./srtm_data"
os.makedirs(output_dir, exist_ok=True)

# Download the files
for granule in results:
    earthaccess.download(granule, output_dir=output_dir)

print(f"Downloaded {len(results)} SRTM DEM files to {output_dir}")

## add step / print statement to confirm download and confirm that the bounding box is the correct study area?? 

## Step 2: Load the additional DEMs you wish to work with 

As an example, we provide a DEM produced by 1954 Chilean military flights (IGM_1954) and a DEM derived from uncrewed aerial vehicle (UAV) flights over Nevados de Chillán in March, 2024 (UAV_2024). You can manually upload your DEMs to the same folder or route to a local file on your computer. Alternatively, you can modify the code in Step 1 to pull other DEM products from online platforms (e.g. IceSAT/-2, ArcticDEM, etc.). 

# code 

## Step 3: Perfom DEM co-registration 
Here we follow the workflow provided by Shean et al. (2023), derived from the Nuth and Kääb (2011) method, here: https://github.com/dshean/demcoreg/blob/master/docs/beginners_doc.md
NB: users should confirm that the co-registration workflow has not been modified since the date of this code publication. 

### Co-registration setup: 
Loading the DEMs, reprojecting the DEMs to the same spatial reference system, extent, and cell size, and calculating slope and aspect to use in the co-registration process:

In [ ]:
## issue... I did all this in the command line so idk how to translate into a python notebook... 